# Análisis exploratorio y descripción general de datos (RawData)

## Introducción

(Añadir descripción de cómo usar DataFiltering)

## 0. Cargar librerías

In [2]:
import pandas as pd
import numpy as np
import time as time
import dask.dataframe as dd
from tqdm import tqdm

import functions as fn

## 1. Cargar el dataset de fallecidos por COVID-19

### Cargar el dataset

In [3]:
url_fal = 'RawData/fallecidos_covid.csv'
df_fal = pd.read_csv(url_fal, sep = ";", encoding= "utf8", low_memory=False)

del url_fal
df_fal.head()

,FECHA_CORTE,FECHA_FALLECIMIENTO,EDAD_DECLARADA,SEXO,CLASIFICACION_DEF,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,id_persona
0,20220106,20210423,41,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,150143,24769779.0
1,20220106,20210607,31,FEMENINO,Criterio SINADEF,CALLAO,CALLAO,CALLAO,070101,24742499.0
2,20220106,20210708,27,FEMENINO,Criterio SINADEF,LIMA,LIMA,CARABAYLLO,150106,24742552.0
3,20220106,20210518,62,MASCULINO,Criterio virolÃ³gico,JUNIN,TARMA,TARMA,120701,24633504.0
4,20220106,20210519,74,MASCULINO,Criterio virolÃ³gico,LIMA,LIMA,SANTA ANITA,150137,24633522.0


### Información general del dataset

In [28]:
df_fal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202867 entries, 0 to 202866
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   FECHA_CORTE          202867 non-null  int64  
 1   FECHA_FALLECIMIENTO  202867 non-null  int64  
 2   EDAD_DECLARADA       202867 non-null  int64  
 3   SEXO                 202867 non-null  object 
 4   CLASIFICACION_DEF    202867 non-null  object 
 5   DEPARTAMENTO         202867 non-null  object 
 6   PROVINCIA            202862 non-null  object 
 7   DISTRITO             202862 non-null  object 
 8   UBIGEO               202867 non-null  object 
 9   id_persona           199466 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 15.5+ MB


### Porcentaje de datos perdidos de cada variable

In [29]:
fn.missing_values(df_fal)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,FECHA_FALLECIMIENTO,0.000000
2,EDAD_DECLARADA,0.000000
3,SEXO,0.000000
4,CLASIFICACION_DEF,0.000000
5,DEPARTAMENTO,0.000000
6,PROVINCIA,0.002465
7,DISTRITO,0.002465
8,UBIGEO,0.000000
9,id_persona,1.676468


### Analizar cada variable del dataset de fallecidos

#### 1.1. FECHA_CORTE

In [3]:
print('Fecha de corte:')
df_fal['FECHA_CORTE'].value_counts().sort_index()

Fecha de corte:


20220103    202867
Name: FECHA_CORTE, dtype: int64

Todos los valores son iguales. La misma fecha de corte para n número de casos, variable no necesaria.

#### 1.2. FECHA_FALLECIMIENTO

In [4]:
# Fecha del primer fallecido reportado
date_a = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_a = date_a.min()
print(f'Fecha del primer fallecido reportado: {date_a}')
del date_a

# Fecha del último fallecido reportado
date_b = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d').dt.date
date_b = date_b.max()
print(f'Fecha del último fallecido reportado: {date_b}')
del date_b

Fecha del primer fallecido reportado: 2020-03-03
Fecha del último fallecido reportado: 2022-01-06


#### 1.3. EDAD_DECLARADA

In [5]:
fal_edad = df_fal['EDAD_DECLARADA'].value_counts()

print('Edad (años): no_casos \nDesde la edad más común de fallecimientos a la menos común.\n')
lst = []

for idx, edad in enumerate(fal_edad):
    index = fal_edad.index
    lst.append(f'{index[idx]}: {edad}')
    
chunks = [lst[x:x+20] for x in range(0, len(lst), 20)]
df = pd.DataFrame.from_records(chunks).transpose()

print(df.to_string(index=False, header=False))

del fal_edad, chunks, df, lst, edad, idx, index

Edad (años): no_casos 
Desde la edad más común de fallecimientos a la menos común.

66: 5513 80: 4150 48: 2225 34: 642   1: 149 102: 40
67: 5414 79: 4028 89: 2090 33: 630  20: 113   5: 40
65: 5389 58: 4027 47: 2029 95: 566  19: 111   6: 38
68: 5360 57: 4009 90: 1876 32: 557 100: 106 103: 22
70: 5260 82: 3860 46: 1856 96: 459   18: 89 104: 16
72: 5201 81: 3831 45: 1781 31: 435    2: 83  106: 9
73: 5187 56: 3719 44: 1676 30: 423   17: 77  108: 6
69: 5166 55: 3698 43: 1506 29: 346   16: 75  105: 5
64: 5099 83: 3567 91: 1447 27: 318  101: 71  107: 3
74: 5089 54: 3443 42: 1359 28: 313    4: 63   -9: 3
71: 5069 84: 3406 92: 1355 97: 306   15: 56  113: 2
63: 5060 85: 3216 41: 1271 26: 267   14: 53  110: 2
61: 5048 53: 3114 40: 1097 98: 246   11: 52  109: 2
75: 4946 86: 2851 39: 1066 25: 243    8: 51  117: 1
62: 4915 52: 2813 93: 1019 24: 204    3: 51  114: 1
76: 4811 51: 2710  38: 976 99: 192    9: 51    None
60: 4602 87: 2673  37: 835 23: 190   10: 50    None
77: 4599 50: 2568  94: 765 21: 1

Existen valores negativos, necesario correguir. Edad mínima 0 años y edad máxima 117 años.

#### 1.4. SEXO

In [6]:
fal_sexo = df_fal['SEXO'].value_counts().sort_index()

for idx, sex in enumerate(fal_sexo):
    index = fal_sexo.index
    print(f'{index[idx]}: {sex}')

del fal_sexo, sex, idx, index

.: 3
FEMENINO: 73775
MASCULINO: 129203


Hay puntos en lugar de vacíos.

#### 1.5. CLASIFICACION_DEF

In [34]:
fal_cla = df_fal['CLASIFICACION_DEF'].value_counts().sort_index()

for idx, clase in enumerate(fal_cla):
    index = fal_cla.index
    print(f'{index[idx]}: {clase}')

del fal_cla, clase, idx, index

Criterio SINADEF: 57905
Criterio clÃ­nico: 9740
Criterio investigaciÃ³n EpidemiolÃ³gica: 16041
Criterio nexo epidemiolÃ³gico: 2947
Criterio radiolÃ³gico: 6716
Criterio serolÃ³gico: 44613
Criterio virolÃ³gico: 64905


#### 1.6. DEPARTAMENTO

In [35]:
fal_dep = df_fal['DEPARTAMENTO'].value_counts().sort_index()

for idx, dep in enumerate(fal_dep):
    index = fal_dep.index
    print(f'{index[idx]}: {dep}')
print(f'\nTotal fallecidos: {fal_dep.sum()}')

del fal_dep, dep, idx, index

AMAZONAS: 1289
ANCASH: 6773
APURIMAC: 1528
AREQUIPA: 9770
AYACUCHO: 2164
CAJAMARCA: 4194
CALLAO: 10123
CUSCO: 4836
HUANCAVELICA: 1185
HUANUCO: 2725
ICA: 8617
JUNIN: 7108
LA LIBERTAD: 10405
LAMBAYEQUE: 8692
LIMA: 89565
LORETO: 4283
MADRE DE DIOS: 777
MOQUEGUA: 1538
PASCO: 1056
PIURA: 12287
PUNO: 4269
SAN MARTIN: 3044
TACNA: 1981
TUMBES: 1596
UCAYALI: 3062

Total fallecidos: 202867


#### 1.7. PROVINCIAS

In [36]:
fal_pro = df_fal['PROVINCIA'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_pro)} provincias.')
del fal_pro

Igual que la variable departamento pero con 196 provincias.


#### 1.8. DISTRITO

In [37]:
fal_dis = df_fal['DISTRITO'].value_counts().sort_index()
print(f'Igual que la variable departamento pero con {len(fal_dis)} distritos.')
del fal_dis

Igual que la variable departamento pero con 1647 distritos.


#### 1.9. UBIGEO

In [38]:
fal_ubi = df_fal['UBIGEO'].value_counts().sort_index()
print(f'{len(fal_ubi)} códigos de UBIGEO, para localizar zonas.')
del fal_ubi

1782 códigos de UBIGEO, para localizar zonas.


#### 1.10. id_persona

In [39]:
df_fal['id_persona'] = df_fal['id_persona'].fillna(0).apply(np.int64)


print('ID únicos para identificar a cada persona: ')
print(df_fal['id_persona'].head().to_string(header=False))

ID únicos para identificar a cada persona: 
0    24804325
1    24652285
2    24652408
3    24652744
4    24803428


In [40]:
del df_fal

## 2. Cargar el dataset de vacunados contra COVID-19

### Cargar usando dask, debido al tamaño del csv

In [41]:
url_vac = 'RawData/TB_VACUNACION_COVID19.csv'

start = time.time()
df_vac = dd.read_csv(url_vac, sep = ",", dtype = {'fecha_vacunacion':'int32', 'id_eess': 'int32',
                                                  'dosis': 'category','id_centro_vacunacion': 'category',
                                                  'id_vacuna': 'category','id_grupo_riesgo': 'category', 
                                                  'edad':'float64'})
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

del url_vac, start, end
df_vac.head()

Read csv with dask:  0.11958074569702148 sec


,id_persona,id_vacunados_covid19,fecha_vacunacion,id_eess,id_centro_vacunacion,id_vacuna,id_grupo_riesgo,dosis,edad
0,7525299,22315832,20210920,14518,2664,6,68,1,38.0
1,5535776,16020164,20210715,20298,0,5,67,1,47.0
2,8837098,25810259,20211001,15366,3861,6,68,2,37.0
3,5222654,15274372,20210610,3320,519,5,54,1,61.0
4,5168206,15456307,20210814,770,250,5,67,2,44.0


### Porcentaje de datos perdidos de cada variable

In [42]:
fn.missing_values(df_vac)

,index,percent_missing
0,0,0.00000
1,1,0.00000
2,2,0.00000
3,3,0.00000
4,4,0.00000
5,5,0.00000
6,6,0.00000
7,7,0.00000
8,8,0.00833


### Analizar cada variable del dataset de vacunados

#### 2.1. id_persona

In [43]:
print('ID únicos para identificar a cada persona:')
df_vac['id_persona'].head()

ID únicos para identificar a cada persona:


0    7525299
1    5535776
2    8837098
3    5222654
4    5168206
Name: id_persona, dtype: int64

#### 2.2. id_vacunados_covid19

In [44]:
print('ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):')
df_vac['id_vacunados_covid19'].head()

ID únicos para identificar a cada vacunado (como un contador de cada vacuna aplicada):


0    22315832
1    16020164
2    25810259
3    15274372
4    15456307
Name: id_vacunados_covid19, dtype: int64

#### 2.3. fecha_vacunacion

In [45]:
date_a = df_vac['fecha_vacunacion'].min().compute()
date_a = pd.to_datetime(date_a, format = '%Y%m%d')

date_b = df_vac['fecha_vacunacion'].max().compute()
date_b = pd.to_datetime(date_b, format = '%Y%m%d')

print(f'Fecha de inicio de vacunación: {date_a} \nFecha última vacunación: {date_b}')
del date_a, date_b

Fecha de inicio de vacunación: 2020-04-27 00:00:00 
Fecha última vacunación: 2022-01-04 00:00:00


#### 2.4. id_eess

In [46]:
print(f'id_iess\tnumber')
df_vac.id_eess.value_counts().compute().sort_index()

id_iess	number


33          36
49         220
51        1419
52        2177
53          63
         ...  
28772      731
28780       23
28791      611
28806    11330
28820      691
Name: id_eess, Length: 7884, dtype: int64

#### 2.5. id_centro_vacunacion	

In [47]:
print(f'id_centro_vacunacion\tnumber')
df_vac.id_centro_vacunacion.value_counts().compute().sort_index()

id_centro_vacunacion	number


0       14741687
1           1924
10         60439
100         3299
104          544
          ...   
2705           2
2998           1
567            1
1865           1
4164           1
Name: id_centro_vacunacion, Length: 3441, dtype: int64

#### 2.6. id_vacuna

In [48]:
print(f'id_vacuna\tnumber')
df_vac.id_vacuna.value_counts().compute().sort_index()

id_vacuna	number


1     3167931
2      371456
3      396814
4      700871
5    27088772
6    18621506
Name: id_vacuna, dtype: int64

#### 2.7. id_grupo_riesgo

In [49]:
df_vac.id_grupo_riesgo.value_counts().compute().sort_index()

0        3909
11     227006
12     334538
17    1535775
22     111776
23     417903
26       4044
36      40384
4       30392
40     345466
45      16996
47      18405
50      17525
51       9826
52       1183
53      18355
54    9504978
56      29077
57     106243
58     110676
59       3063
6        2297
60      30110
61      11345
62    6048890
63      36509
64      70757
65      44765
66     255997
67    7498829
68    8157747
69     130656
7      316962
70    1671237
71    8413330
72       4967
73       4814
74       3228
75       4688
76      17600
77    4732210
78        943
79         82
1         207
24       1660
Name: id_grupo_riesgo, dtype: int64

#### 2.8. dosis

In [50]:
print(f'dosis\tnumber')
df_vac.dosis.value_counts().compute()

dosis	number


1    24511772
2    21611587
3     4223869
4         116
5           6
Name: dosis, dtype: int64

#### 2.9. edad

In [51]:
df_vac.edad.value_counts().compute().sort_index()

0.0      3
1.0      2
2.0      4
3.0      2
4.0      7
        ..
141.0    2
143.0    2
146.0    2
147.0    1
224.0    2
Name: edad, Length: 136, dtype: int64

## 3. Cargar el dataset de casos positivos de COVID-19

### Cargar el dataset

In [52]:
url_cas = 'RawData/positivos_covid.csv'
df_cas = pd.read_csv('RawData/positivos_covid.csv', sep = ';')

del url_cas
df_cas.head()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
0,20220103,TACNA,TACNA,CORONEL GREGORIO ALBARRACIN L.,AG,16.0,MASCULINO,20210109.0,230110.0,24833983.0
1,20220103,MADRE DE DIOS,TAMBOPATA,TAMBOPATA,PR,26.0,FEMENINO,20200829.0,170101.0,24833731.0
2,20220103,CALLAO,CALLAO,LA PERLA,PCR,27.0,MASCULINO,20210217.0,70104.0,24833734.0
3,20220103,LIMA,EN INVESTIGACIÓN,EN INVESTIGACIÓN,PCR,39.0,FEMENINO,20200724.0,NaN,24833739.0
4,20220103,LIMA,LIMA,VILLA MARIA DEL TRIUNFO,PR,26.0,MASCULINO,20200821.0,150143.0,24833746.0


### Porcentaje de datos perdidos de cada variable

In [53]:
fn.missing_values(df_cas)

,index,percent_missing
0,FECHA_CORTE,0.000000
1,DEPARTAMENTO,0.000000
2,PROVINCIA,0.000000
3,DISTRITO,0.000000
4,METODODX,0.000000
5,EDAD,0.015023
6,SEXO,0.000043
7,FECHA_RESULTADO,0.087581
8,UBIGEO,4.879828
9,id_persona,1.756993


### Analizar cada variable del dataset de casos reportados